In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings 
warnings.filterwarnings('ignore')
import os 
import re
import FinanceDataReader as fdr
import time

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, roc_auc_score ,accuracy_score, precision_score, recall_score, confusion_matrix
import joblib 

rfc = RandomForestClassifier()
xgb = XGBClassifier()
cat = CatBoostClassifier()
models = [rfc,xgb,cat]

# 함수

## corp_code

In [3]:
df_kospi = pd.read_csv('./data/recent_kospi_list.csv',index_col=0)
# 코드 반환
def corp_code(corp_name):
    code = df_kospi[df_kospi['Name']==corp_name]['Symbol'].iloc[0]
    code = str(code).zfill(6)
    return code

## stock_price()

In [4]:
# 주가데이터
def stock_price(code,bgn_date= '2016-01-01',end_date= '2022-03-31'):

    df_p = fdr.DataReader(code,bgn_date,end_date)
    df_p = df_p.reset_index()
    df_p = df_p.rename(columns = {'Date': '날짜'})
    df_p = df_p.set_index('날짜',drop=True)
    
    return df_p

## merge()

In [5]:
# merge
def merge(df_count,df_p):
    df_merge = pd.merge(df_count,df_p['Close'],left_index=True,right_index=True, how='right')
    return df_merge

## mscaler()

In [6]:
from sklearn.preprocessing import MinMaxScaler

def mscaler(df):

    df.index = pd.DatetimeIndex(df.index)
    col_list = df.columns
    scaler = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[col_list] = scaler.fit_transform(df[col_list])
    
    return df_scaled

## add_target()

In [7]:
def add_target(df_merge,window_size, period_rate):

    earning_lst = []
    for i in range(len(df_merge)-window_size):
        
        earning_rate = (df_merge.iloc[i+window_size,-1]- df_merge.iloc[i,-1])/df_merge.iloc[i,-1]

        if earning_rate >= period_rate:
            earning_lst.append(1)
        else:
            earning_lst.append(0)

    df_model = df_merge.iloc[:-window_size,:]
    df_model['target'] = earning_lst
    
    return df_model

## feature_visualization()

In [8]:
def feature_visualization(save_path,num):
    
    model = joblib.load(save_path)
    # 배열형태로 반환
    ft_importance_values = model.feature_importances_

    # 정렬과 시각화를 쉽게 하기 위해 series 전환
    ft_series = pd.Series(ft_importance_values, index = x_test.columns)
    ft_top20 = ft_series.sort_values(ascending=False)[:num]

    # 시각화
    plt.figure(figsize=(8,6))
    plt.title('Feature Importance Top 20')
    sns.barplot(x=ft_top20, y=ft_top20.index)
    plt.show()
    
    return ft_top20

In [9]:
pd.set_option('display.max.colwidth', 70)

# 결과 정리

## 모든파일 만들기 7일 5%

In [29]:
window_size = 7
period_rate= 0.05

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[14:25:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[14:25:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[14:25:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[14:25:43] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[14:26:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[14:26:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[14:26:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[14:26:48] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[14:26:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[14:26:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

회사이름:  한라홀딩스
[14:47:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미글로벌
[14:47:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미반도체
[14:47:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미사이언스
[14:47:44] WARNIN

회사이름:  한일홀딩스
[14:48:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전KPS
[14:48:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전KPS
모델이름:  CatBoostClassifier
accuracy:0.7937, precision:0.6667, recall:0.0741, roc_auc:0.5084
./data/machine_model3_7일_0.05/한전KPS_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  한전산업
[14:48:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0,

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[14:49:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[14:49:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[14:49:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
모델이름:  XGBClassi

회사이름:  후성
[14:50:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휠라홀딩스
[14:50:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휴스틸
[14:50:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  흥아해운
[14:50:40] WARNING: C:/U

In [30]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,CJ CGV,CatBoostClassifier,0.809524,0.666667,0.080000,0.511683,./data/machine_model3_7일_0.05/CJ CGV_CatBoostClassifier.pkl
1,고려아연,CatBoostClassifier,0.793651,0.666667,0.074074,0.364759,./data/machine_model3_7일_0.05/고려아연_CatBoostClassifier.pkl
2,금호에이치티,RandomForestClassifier,0.801587,0.600000,0.115385,0.504231,./data/machine_model3_7일_0.05/금호에이치티_RandomForestClassifier.pkl
3,금호에이치티,XGBClassifier,0.801587,0.600000,0.115385,0.516346,./data/machine_model3_7일_0.05/금호에이치티_XGBClassifier.pkl
4,금호에이치티,CatBoostClassifier,0.801587,0.666667,0.076923,0.494231,./data/machine_model3_7일_0.05/금호에이치티_CatBoostClassifier.pkl
5,대교,RandomForestClassifier,0.817460,0.666667,0.083333,0.507353,./data/machine_model3_7일_0.05/대교_RandomForestClassifier.pkl
6,두산,RandomForestClassifier,0.539683,0.571429,0.067797,0.503668,./data/machine_model3_7일_0.05/두산_RandomForestClassifier.pkl
7,무학,RandomForestClassifier,0.761905,0.833333,0.147059,0.594150,./data/machine_model3_7일_0.05/무학_RandomForestClassifier.pkl
8,무학,XGBClassifier,0.753968,0.666667,0.176471,0.588075,./data/machine_model3_7일_0.05/무학_XGBClassifier.pkl
9,범양건영,XGBClassifier,0.714286,0.666667,0.054054,0.547981,./data/machine_model3_7일_0.05/범양건영_XGBClassifier.pkl


In [ ]:
model

In [23]:
# 모델 검증하기
# 한진칼,  
corp_name = '대웅'
model_path = './data/machine_model2_30일_0.1/대웅_CatBoostClassifier.pkl'
window_size = 30
period_rate= 0.1

code = corp_code(corp_name)
df_p = stock_price(code)

path = './data/데이터_뉴스키워드빈도/'
file_path = os.path.join(path,corp_name+'.csv')
df_count = pd.read_csv(file_path,index_col=0)
df_count.index = pd.DatetimeIndex(df_count.index)

df_merge = merge(df_count,df_p)
df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

x_train = df_model.loc[:'2021-10-01',:'Close']
y_train = df_model.loc[:'2021-10-01','target']

x_test = df_model.loc['2021-10-01':,:'Close']
y_test = df_model.loc['2021-10-01':,'target']


# featurn importance
model = joblib.load(model_path)
pred = model.predict(x_test)

print(classification_report(y_test,pred))


              precision    recall  f1-score   support

           0       0.92      0.93      0.93        74
           1       0.71      0.67      0.69        18

    accuracy                           0.88        92
   macro avg       0.81      0.80      0.81        92
weighted avg       0.88      0.88      0.88        92



## 모든파일 만들기 7일 10%

In [31]:
window_size = 7
period_rate= 0.1

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[15:15:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[15:15:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[15:15:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[15:15:32] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[15:16:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[15:16:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[15:16:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[15:16:38] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[15:16:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[15:16:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

회사이름:  한라홀딩스
[15:36:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미글로벌
[15:36:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미반도체
[15:36:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미사이언스
[15:37:02] WARNIN

회사이름:  한전KPS
[15:37:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전산업
[15:37:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진
[15:37:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진중공업홀딩스
[15:38:04] WARNING:

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[15:38:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[15:38:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[15:38:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대백화점
[15:38:50] WARNIN

회사이름:  후성
[15:39:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휠라홀딩스
[15:39:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휴스틸
[15:39:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  흥아해운
[15:39:54] WARNING: C:/U

In [32]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,OCI,XGBClassifier,0.841270,0.571429,0.190476,0.629932,./data/machine_model3_7일_0.1/OCI_XGBClassifier.pkl
1,SG세계물산,RandomForestClassifier,0.976190,0.666667,0.500000,0.745902,./data/machine_model3_7일_0.1/SG세계물산_RandomForestClassifier.pkl
2,STX중공업,XGBClassifier,0.880952,0.750000,0.176471,0.592822,./data/machine_model3_7일_0.1/STX중공업_XGBClassifier.pkl
3,대유플러스,CatBoostClassifier,0.746032,0.666667,0.060606,0.611763,./data/machine_model3_7일_0.1/대유플러스_CatBoostClassifier.pkl
4,동국제강,CatBoostClassifier,0.873016,0.750000,0.166667,0.645319,./data/machine_model3_7일_0.1/동국제강_CatBoostClassifier.pkl
5,이수화학,XGBClassifier,0.833333,0.666667,0.090909,0.588505,./data/machine_model3_7일_0.1/이수화학_XGBClassifier.pkl
6,일성건설,RandomForestClassifier,0.698413,0.750000,0.075000,0.560610,./data/machine_model3_7일_0.1/일성건설_RandomForestClassifier.pkl
7,일성건설,XGBClassifier,0.698413,0.600000,0.150000,0.613372,./data/machine_model3_7일_0.1/일성건설_XGBClassifier.pkl
8,주연테크,XGBClassifier,0.880952,0.666667,0.125000,0.526136,./data/machine_model3_7일_0.1/주연테크_XGBClassifier.pkl
9,한신공영,CatBoostClassifier,0.936508,0.666667,0.222222,0.616334,./data/machine_model3_7일_0.1/한신공영_CatBoostClassifier.pkl


## 모든파일 만들기 7일 15%

In [10]:
window_size = 7
period_rate= 0.15

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[16:27:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[16:27:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[16:27:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[16:27:25] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[16:28:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[16:28:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[16:28:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[16:28:28] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[16:28:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[16:28:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

회사이름:  한라홀딩스
[16:48:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미글로벌
[16:48:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미반도체
[16:48:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미사이언스
[16:48:49] WARNIN

회사이름:  한전KPS
[16:49:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전산업
[16:49:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진
[16:49:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진중공업홀딩스
[16:49:53] WARNING:

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[16:50:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[16:50:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[16:50:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대백화점
[16:50:39] WARNIN

회사이름:  후성
[16:51:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휠라홀딩스
[16:51:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  휴스틸
[16:51:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  흥아해운
[16:51:42] WARNING: C:/U

In [11]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,대유플러스,CatBoostClassifier,0.777778,0.750000,0.100000,0.559896,./data/machine_model3_7일_0.15/대유플러스_CatBoostClassifier.pkl
1,동국제강,CatBoostClassifier,0.920635,0.750000,0.250000,0.669591,./data/machine_model3_7일_0.15/동국제강_CatBoostClassifier.pkl
2,에넥스,RandomForestClassifier,0.912698,0.666667,0.166667,0.578216,./data/machine_model3_7일_0.15/에넥스_RandomForestClassifier.pkl


## 모든파일 만들기 30일 5%

In [12]:
window_size = 30
period_rate= 0.05

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[16:55:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[16:55:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[16:55:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[16:55:14] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[16:56:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[16:56:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[16:56:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[16:56:22] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[16:56:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[16:56:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[17:17:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한솔홀딩스
모델이름:  CatBoostClassifier
accuracy:0.8254, precision:0.7500, recall:0.1250, roc_auc:0.5229
./data/machine_model3_30일_0.05/한솔홀딩스_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  한솔홈데코
[17:17:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한신공영
[17:17:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default

회사이름:  한화에어로스페이스
[17:18:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한화에어로스페이스
모델이름:  XGBClassifier
accuracy:0.5317, precision:0.6667, recall:0.0968, roc_auc:0.4525
./data/machine_model3_30일_0.05/한화에어로스페이스_XGBClassifier.pkl
--------------------------------------------------
회사이름:  해태제과식품
[17:18:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대건설
[17:18:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[17:18:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[17:18:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[17:18:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대백화점
[17:18:37] WARNIN

회사이름:  효성
모델이름:  CatBoostClassifier
accuracy:0.6429, precision:0.5500, recall:0.4490, roc_auc:0.5879
./data/machine_model3_30일_0.05/효성_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  효성ITX
[17:19:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  후성
회사이름:  후성
모델이름:  RandomForestClassifier
accuracy:0.3571, precision:0.7500, recall:0.1648, roc_auc:0.5171
./data/machine_model3_30일_0.05/후성_RandomForestClassifier.pkl
--------------------------------------------------
[17:19:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. E

In [13]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,DB,RandomForestClassifier,0.642857,0.694444,0.423729,0.628889,./data/machine_model3_30일_0.05/DB_RandomForestClassifier.pkl
1,DB,XGBClassifier,0.642857,0.684211,0.440678,0.648748,./data/machine_model3_30일_0.05/DB_XGBClassifier.pkl
2,DB,CatBoostClassifier,0.619048,0.689655,0.338983,0.660764,./data/machine_model3_30일_0.05/DB_CatBoostClassifier.pkl
3,DI동일,RandomForestClassifier,0.420635,0.600000,0.040541,0.521440,./data/machine_model3_30일_0.05/DI동일_RandomForestClassifier.pkl
4,DI동일,XGBClassifier,0.420635,0.600000,0.040541,0.523259,./data/machine_model3_30일_0.05/DI동일_XGBClassifier.pkl
...,...,...,...,...,...,...,...
143,화신,CatBoostClassifier,0.428571,0.703704,0.228916,0.542169,./data/machine_model3_30일_0.05/화신_CatBoostClassifier.pkl
144,효성,CatBoostClassifier,0.642857,0.550000,0.448980,0.587861,./data/machine_model3_30일_0.05/효성_CatBoostClassifier.pkl
145,후성,RandomForestClassifier,0.357143,0.750000,0.164835,0.517111,./data/machine_model3_30일_0.05/후성_RandomForestClassifier.pkl
146,후성,XGBClassifier,0.301587,0.588235,0.109890,0.409419,./data/machine_model3_30일_0.05/후성_XGBClassifier.pkl


## 모든파일 만들기 30일 10%

In [9]:
window_size = 30
period_rate= 0.1

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[17:40:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[17:40:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[17:40:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[17:40:38] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[17:41:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[17:41:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[17:41:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[17:41:48] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[17:41:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[17:41:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[18:02:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한독
[18:02:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한라홀딩스
[18:02:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미글로벌
[18:02:19] WARNING: C:/Users/Admin

회사이름:  한일철강
[18:03:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한일홀딩스
[18:03:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전KPS
[18:03:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전산업
[18:03:19] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[18:04:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[18:04:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[18:04:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대백화점
[18:04:15] WARNIN

회사이름:  효성ITX
[18:05:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  후성
회사이름:  후성
모델이름:  RandomForestClassifier
accuracy:0.4286, precision:0.6000, recall:0.0411, roc_auc:0.5368
./data/machine_model3_30일_0.1/후성_RandomForestClassifier.pkl
--------------------------------------------------
[18:05:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  후성
모델이름:  CatBoostClassifier
accuracy:0.4365, precision:0.7500, recall:0.0411, roc_auc:0.4572
./data/machine_model3_30일_0.1/후성_CatBoos

In [10]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,CJ,CatBoostClassifier,0.825397,0.666667,0.086957,0.581891,./data/machine_model3_30일_0.1/CJ_CatBoostClass...
1,DB,CatBoostClassifier,0.674603,0.600000,0.071429,0.605159,./data/machine_model3_30일_0.1/DB_CatBoostClass...
2,DI동일,XGBClassifier,0.595238,0.600000,0.057692,0.470894,./data/machine_model3_30일_0.1/DI동일_XGBClassifi...
3,DI동일,CatBoostClassifier,0.595238,0.666667,0.038462,0.546518,./data/machine_model3_30일_0.1/DI동일_CatBoostCla...
4,SBS,RandomForestClassifier,0.531746,0.650000,0.366197,0.551216,./data/machine_model3_30일_0.1/SBS_RandomForest...
...,...,...,...,...,...,...,...
60,한진중공업홀딩스,RandomForestClassifier,0.714286,0.666667,0.054054,0.558002,./data/machine_model3_30일_0.1/한진중공업홀딩스_RandomF...
61,화신,XGBClassifier,0.452381,0.545455,0.169014,0.483611,./data/machine_model3_30일_0.1/화신_XGBClassifier...
62,화신,CatBoostClassifier,0.444444,0.529412,0.126761,0.464661,./data/machine_model3_30일_0.1/화신_CatBoostClass...
63,후성,RandomForestClassifier,0.428571,0.600000,0.041096,0.536831,./data/machine_model3_30일_0.1/후성_RandomForestC...


## 모든파일 만들기 30일 15%

In [9]:
window_size = 30
period_rate= 0.15

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:] :
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc[:'2021-04-01',:last_col]
        y_train = df_model.loc[:'2021-04-01','target']
        
        x_test = df_model.loc['2021-04-01':'2021-10-01',:last_col]
        y_test = df_model.loc['2021-04-01':'2021-10-01','target']
        
        x_val = df_model.loc['2021-10-01':,:last_col]
        y_val = df_model.loc['2021-10-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_test)
            accuracy = accuracy_score(y_test, pred)
            proba = model.predict_proba(x_test)[:,1]
            precision = precision_score(y_test, pred)
            recall = recall_score(y_test, pred)

            try:
                roc_auc = roc_auc_score(y_test, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{window_size}일_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_test/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[20:07:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[20:07:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[20:07:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[20:07:07] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[20:08:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[20:08:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[20:08:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[20:08:14] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[20:08:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[20:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

회사이름:  한라홀딩스
[20:28:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미글로벌
[20:28:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미반도체
[20:28:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한미사이언스
[20:28:50] WARNIN

회사이름:  한전KPS
[20:29:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한전산업
[20:29:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진
[20:29:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한진중공업홀딩스
[20:29:54] WARNING:

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[20:30:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[20:30:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
[20:30:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대백화점
[20:30:39] WARNIN

회사이름:  후성
회사이름:  후성
모델이름:  RandomForestClassifier
accuracy:0.4841, precision:0.6667, recall:0.0303, roc_auc:0.4957
./data/machine_model3_30일_0.15/후성_RandomForestClassifier.pkl
--------------------------------------------------
[20:31:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  후성
모델이름:  CatBoostClassifier
accuracy:0.4841, precision:0.6667, recall:0.0303, roc_auc:0.4980
./data/machine_model3_30일_0.15/후성_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  휠라홀딩스
[20:31:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. E

In [11]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,DB,CatBoostClassifier,0.809524,0.666667,0.080000,0.612277,./data/machine_model3_30일_0.15/DB_CatBoostClas...
1,LG,RandomForestClassifier,0.944444,0.750000,0.333333,0.629155,./data/machine_model3_30일_0.15/LG_RandomForest...
2,SBS,RandomForestClassifier,0.698413,0.655172,0.404255,0.616213,./data/machine_model3_30일_0.15/SBS_RandomFores...
3,SBS,XGBClassifier,0.690476,0.625000,0.425532,0.612847,./data/machine_model3_30일_0.15/SBS_XGBClassifi...
4,SBS,CatBoostClassifier,0.674603,0.588235,0.425532,0.619311,./data/machine_model3_30일_0.15/SBS_CatBoostCla...
5,SKC,CatBoostClassifier,0.785714,0.571429,0.142857,0.485787,./data/machine_model3_30일_0.15/SKC_CatBoostCla...
6,SK가스,CatBoostClassifier,0.698413,0.666667,0.051282,0.476569,./data/machine_model3_30일_0.15/SK가스_CatBoostCl...
7,두산,XGBClassifier,0.603175,0.875000,0.125000,0.490179,./data/machine_model3_30일_0.15/두산_XGBClassifie...
8,롯데관광개발,RandomForestClassifier,0.865079,0.666667,0.111111,0.569444,./data/machine_model3_30일_0.15/롯데관광개발_RandomFo...
9,삼성공조,CatBoostClassifier,0.603175,0.619048,0.236364,0.594366,./data/machine_model3_30일_0.15/삼성공조_CatBoostCl...


In [13]:
y_test

날짜
2021-10-01    0
2021-10-05    0
2021-10-06    0
2021-10-07    0
2021-10-08    0
             ..
2022-02-09    1
2022-02-10    1
2022-02-11    1
2022-02-14    1
2022-02-15    1
Name: target, Length: 92, dtype: int64

## 모든파일 만들기 3개월 5%

In [24]:
month = 3
window_size = 21*month
period_rate= 0.05

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-07-01',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-07-01','target']
        
        x_test = df_model.loc['2021-07-01':,:last_col]
        y_test = df_model.loc['2021-07-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[16:31:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  AJ네트웍스
모델이름:  CatBoostClassifier
accuracy:0.2984, precision:0.5714, recall:0.0455, roc_auc:0.4009
./data/machine_model3_3개월_0.05/AJ네트웍스_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  AK홀딩스
[16:31:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[16:31:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
[16:32:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0,

회사이름:  DSR
모델이름:  XGBClassifier
accuracy:0.3629, precision:0.6000, recall:0.1098, roc_auc:0.4746
./data/machine_model3_3개월_0.05/DSR_XGBClassifier.pkl
--------------------------------------------------
회사이름:  E1
회사이름:  E1
모델이름:  RandomForestClassifier
accuracy:0.1855, precision:0.5385, recall:0.0686, roc_auc:0.3333
./data/machine_model3_3개월_0.05/E1_RandomForestClassifier.pkl
--------------------------------------------------
[16:32:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  F&F홀딩스
[16:32:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicit

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

...

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대로템
모델이름:  CatBoostClassifier
accuracy:0.4032, precision:0.5833, recall:0.0921, roc_auc:0.5471
./data/machine_model3_3개월_0.05/현대로템_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대리바트
회사이름:  현대리바트
모델이름:  RandomForestClassifier
accuracy:0.4032, precision:0.8000, recall:0.0519, roc_auc:0.5684
./data/machine_model3_3개월_0.05/현대리바트_RandomForestClassifier.pkl
--------------------------------------------------
[16:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대리바트
모델이름:  XGBClassifier
accuracy:0.3952, precision:0.7500, recall:0.0390, roc_auc:0.5134
./data/machine_model3_3개월_0.05/현대리바트_XGBClassifier.pkl
--------------------------------------------------
회사이름:  현대모비스
[16:56:06] WARNING: C:/

회사이름:  호텔신라
회사이름:  호텔신라
모델이름:  RandomForestClassifier
accuracy:0.4919, precision:0.5111, recall:0.3594, roc_auc:0.4673
./data/machine_model3_3개월_0.05/호텔신라_RandomForestClassifier.pkl
--------------------------------------------------
[16:56:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  화승인더
[16:56:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  화승코퍼레이션
[16:56:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoos

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화신
모델이름:  CatBoostClassifier
accuracy:0.2823, precision:0.6585, recall:0.2647, roc_auc:0.2313
./data/machine_model3_3개월_0.05/화신_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  화천기공
[16:56:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
회사이름:  황금에스티
모델이름:  RandomForestClassifier
accuracy:0.7339, precision:0.7436, recall:0.9667, roc_auc:0.5227
./data/machine_model3_3개월_0.05/황금에스티_RandomForestClassifier.pkl
--------------------------------------------------
[16:57:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'log

In [25]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,AJ네트웍스,CatBoostClassifier,0.298387,0.571429,0.045455,0.400884,./data/machine_model3_3개월_0.05/AJ네트웍스_CatBoostClassifier.pkl
1,CJ CGV,RandomForestClassifier,0.540323,0.560000,0.444444,0.531876,./data/machine_model3_3개월_0.05/CJ CGV_RandomForestClassifier.pkl
2,CJ CGV,XGBClassifier,0.508065,0.527778,0.301587,0.583008,./data/machine_model3_3개월_0.05/CJ CGV_XGBClassifier.pkl
3,CJ제일제당,RandomForestClassifier,0.572581,0.615385,0.271186,0.544329,./data/machine_model3_3개월_0.05/CJ제일제당_RandomForestClassifier.pkl
4,CJ제일제당,XGBClassifier,0.548387,0.578947,0.186441,0.582790,./data/machine_model3_3개월_0.05/CJ제일제당_XGBClassifier.pkl
...,...,...,...,...,...,...,...
440,휠라홀딩스,RandomForestClassifier,0.516129,0.520325,0.984615,0.472099,./data/machine_model3_3개월_0.05/휠라홀딩스_RandomForestClassifier.pkl
441,휠라홀딩스,CatBoostClassifier,0.524194,0.524590,0.984615,0.484876,./data/machine_model3_3개월_0.05/휠라홀딩스_CatBoostClassifier.pkl
442,휴스틸,RandomForestClassifier,0.241935,0.666667,0.061856,0.526919,./data/machine_model3_3개월_0.05/휴스틸_RandomForestClassifier.pkl
443,휴스틸,XGBClassifier,0.233871,0.625000,0.051546,0.533792,./data/machine_model3_3개월_0.05/휴스틸_XGBClassifier.pkl


## 모든파일 만들기 3개월 10%

In [26]:
month = 3
window_size = 21*month
period_rate= 0.1

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-07-01',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-07-01','target']
        
        x_test = df_model.loc['2021-07-01':,:last_col]
        y_test = df_model.loc['2021-07-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
회사이름:  AJ네트웍스
모델이름:  RandomForestClassifier
accuracy:0.3790, precision:0.6667, recall:0.0506, roc_auc:0.4115
./data/machine_model3_3개월_0.1/AJ네트웍스_RandomForestClassifier.pkl
--------------------------------------------------
[17:02:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[17:02:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[17:02:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBo

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
회사이름:  GS글로벌
모델이름:  RandomForestClassifier
accuracy:0.4919, precision:0.5714, recall:0.1231, roc_auc:0.5270
./data/machine_model3_3개월_0.1/GS글로벌_RandomForestClassifier.pkl
--------------------------------------------------
[17:03:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS글로벌
모델이름:  XGBClassifier
accuracy:0.4839, precision:0.5714, recall:0.0615, roc_auc:0.5248
./data/machine_model3_3개월_0.1/GS글로벌_XGBClassifier.pkl
--------------------------------------------------
회사이름:  GS글로벌
모델이름:  CatBoostClassifier
accuracy:0.4839, precision:0.5714, recall:0.0615, roc_auc:0.5301
./data/machine_model3_3개월_0.1/GS글로벌_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  GS리테일
[17:03:45] WARNING: C:/U

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[17:03:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
회사이름:  HMM
모델이름:  RandomForestClassifier
accuracy:0.5081, precision:0.6023, recall:0.6709, roc_auc:0.4513
./data/machine_model3_3개월_0.1/HMM_RandomForestClassifier.pkl
--------------------------------------------------
[17:03:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
모델이름:  XGBClassifier
accuracy:0.4597, precision:0.5769, recall:0.5696, roc_auc:0.4094
./data/machine_model3_3개월_0.1/HMM_XGBClas

...

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대로템
모델이름:  CatBoostClassifier
accuracy:0.5000, precision:0.6250, recall:0.0781, roc_auc:0.5186
./data/machine_model3_3개월_0.1/현대로템_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대리바트
[17:28:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[17:29:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
회사이름:  현대미포조선
모델이름:  RandomForestClassifier
accuracy:0.4435, precision:0.5862, recall:0.4304, roc_auc:0.4887
./data/machine_mode

회사이름:  화신
회사이름:  화신
모델이름:  RandomForestClassifier
accuracy:0.4355, precision:0.6981, recall:0.4066, roc_auc:0.4259
./data/machine_model3_3개월_0.1/화신_RandomForestClassifier.pkl
--------------------------------------------------
[17:30:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  화신
모델이름:  XGBClassifier
accuracy:0.2903, precision:0.5517, recall:0.1758, roc_auc:0.4171
./data/machine_model3_3개월_0.1/화신_XGBClassifier.pkl
--------------------------------------------------


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화신
모델이름:  CatBoostClassifier
accuracy:0.3145, precision:0.6000, recall:0.1978, roc_auc:0.3490
./data/machine_model3_3개월_0.1/화신_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  화천기공
[17:30:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
[17:30:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
모델이름:  CatBoostClassifier
accuracy:0.3790, precision:0.6667, recall:0.0256, roc_auc:0.5506
./data/machine_model3_3개월_0.1/황금에스티_CatBoos

In [27]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,AJ네트웍스,RandomForestClassifier,0.379032,0.666667,0.050633,0.411533,./data/machine_model3_3개월_0.1/AJ네트웍스_RandomForestClassifier.pkl
1,DB,RandomForestClassifier,0.524194,0.578947,0.619718,0.489636,./data/machine_model3_3개월_0.1/DB_RandomForestClassifier.pkl
2,DB,XGBClassifier,0.564516,0.626866,0.591549,0.550757,./data/machine_model3_3개월_0.1/DB_XGBClassifier.pkl
3,DB,CatBoostClassifier,0.508065,0.569444,0.577465,0.458278,./data/machine_model3_3개월_0.1/DB_CatBoostClassifier.pkl
4,E1,RandomForestClassifier,0.306452,0.700000,0.077778,0.395752,./data/machine_model3_3개월_0.1/E1_RandomForestClassifier.pkl
...,...,...,...,...,...,...,...
243,효성,CatBoostClassifier,0.298387,0.852941,0.261261,0.448371,./data/machine_model3_3개월_0.1/효성_CatBoostClassifier.pkl
244,휠라홀딩스,RandomForestClassifier,0.508065,0.538462,0.222222,0.513271,./data/machine_model3_3개월_0.1/휠라홀딩스_RandomForestClassifier.pkl
245,휴스틸,RandomForestClassifier,0.258065,0.750000,0.031915,0.489539,./data/machine_model3_3개월_0.1/휴스틸_RandomForestClassifier.pkl
246,휴스틸,XGBClassifier,0.266129,0.800000,0.042553,0.508688,./data/machine_model3_3개월_0.1/휴스틸_XGBClassifier.pkl


## 모든파일 만들기 3개월 15%

In [10]:
month = 3
window_size = 21*month
period_rate= 0.15

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-07-01',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-07-01','target']
        
        x_test = df_model.loc['2021-07-01':,:last_col]
        y_test = df_model.loc['2021-07-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
회사이름:  AJ네트웍스
모델이름:  RandomForestClassifier
accuracy:0.4758, precision:0.6667, recall:0.0303, roc_auc:0.4169
./data/machine_model3_3개월_0.15/AJ네트웍스_RandomForestClassifier.pkl
--------------------------------------------------
[18:27:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[18:27:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[18:27:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGB

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[18:28:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS글로벌
모델이름:  CatBoostClassifier
accuracy:0.5887, precision:0.6667, recall:0.0755, roc_auc:0.6327
./data/machine_model3_3개월_0.15/GS글로벌_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  GS리테일
[18:28:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[18:28:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0,

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[18:28:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
회사이름:  HMM
모델이름:  RandomForestClassifier
accuracy:0.4839, precision:0.5747, recall:0.6494, roc_auc:0.4030
./data/machine_model3_3개월_0.15/HMM_RandomForestClassifier.pkl
--------------------------------------------------
[18:28:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
모델이름:  XGBClassifier
accuracy:0.4355, precision:0.5455, recall:0.5455, roc_auc:0.3906
./data/machine_model3_3개월_0.15/HMM_XGBCl

...

회사이름:  한신공영
모델이름:  CatBoostClassifier
accuracy:0.5000, precision:0.5455, recall:0.0952, roc_auc:0.5546
./data/machine_model3_3개월_0.15/한신공영_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  한신기계
[18:49:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한온시스템
[18:49:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  한올바이오파마
[18:49:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대리바트
[18:50:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[18:50:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
회사이름:  현대미포조선
모델이름:  RandomForestClassifier
accuracy:0.4355, precision:0.5600, recall:0.3684, roc_auc:0.4343
./data/machine_model3_3개월_0.15/현대미포조선_RandomForestClassifier.pkl
--------------------------------------------------
[18:50:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in X

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화천기공
[18:51:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
[18:51:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  효성
회사이름:  효성
모델이름:  RandomForestClassifier
accuracy:0.3790, precision:0.7895, recall:0.1705, roc_auc:0.5565
./data/machine_model3_3개월_0.15/효성_RandomForestClassifier.pkl
--------------------------------------------------
[18:52:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0,

In [29]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,LG이노텍,RandomForestClassifier,0.380165,0.800000,0.142857,0.456725,./data/machine_model3_3개월_0.15/LG이노텍_RandomForestClassifier.pkl
1,LG이노텍,XGBClassifier,0.371901,0.681818,0.178571,0.487130,./data/machine_model3_3개월_0.15/LG이노텍_XGBClassifier.pkl
2,LG이노텍,CatBoostClassifier,0.371901,0.750000,0.142857,0.457046,./data/machine_model3_3개월_0.15/LG이노텍_CatBoostClassifier.pkl
3,LIG넥스원,XGBClassifier,0.347107,0.750000,0.037037,0.560185,./data/machine_model3_3개월_0.15/LIG넥스원_XGBClassifier.pkl
4,TCC스틸,XGBClassifier,0.330579,0.571429,0.048780,0.436210,./data/machine_model3_3개월_0.15/TCC스틸_XGBClassifier.pkl
5,대웅제약,RandomForestClassifier,0.876033,0.666667,0.125000,0.607143,./data/machine_model3_3개월_0.15/대웅제약_RandomForestClassifier.pkl
6,두산,XGBClassifier,0.619835,0.565217,0.265306,0.609410,./data/machine_model3_3개월_0.15/두산_XGBClassifier.pkl
7,두산,CatBoostClassifier,0.603306,0.666667,0.040816,0.480159,./data/machine_model3_3개월_0.15/두산_CatBoostClassifier.pkl
8,디티알오토모티브,RandomForestClassifier,0.173554,0.666667,0.019802,0.473515,./data/machine_model3_3개월_0.15/디티알오토모티브_RandomForestClassifier.pkl
9,디티알오토모티브,CatBoostClassifier,0.173554,0.666667,0.019802,0.491584,./data/machine_model3_3개월_0.15/디티알오토모티브_CatBoostClassifier.pkl


## 모든파일 만들기 6개월 5% 

In [11]:
month = 6
window_size = 21*month
period_rate= 0.05

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-03-31',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-03-31','target']
        
        x_test = df_model.loc['2021-04-01':,:last_col]
        y_test = df_model.loc['2021-04-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[19:06:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[19:06:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
회사이름:  BGF
모델이름:  RandomForestClassifier
accuracy:0.5833, precision:0.5556, recall:0.5357, roc_auc:0.6417
./data/machine_model3_6개월_0.05/BGF_RandomForestClassifier.pkl
--------------------------------------------------
[19:06:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
회사이름:  GS글로벌
모델이름:  RandomForestClassifier
accuracy:0.7667, precision:0.9184, recall:0.8182, roc_auc:0.3509
./data/machine_model3_6개월_0.05/GS글로벌_RandomForestClassifier.pkl
--------------------------------------------------
[19:07:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS글로벌
모델이름:  XGBClassifier
accuracy:0.7667, precision:0.9556, recall:0.7818, roc_auc:0.6564
./data/machine_model3_6개월_0.05/GS글로벌_XGBClassifier.pkl
--------------------------------------------------
회사이름:  GS글로벌
모델이름:  CatBoostClassifier
accuracy:0.8167, precision:0.9400, recall:0.8545, roc_auc:0.5655
./data/machine_model3_6개월_0.05/GS글로벌_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  GS리테일
[19:07:50] WARNING: C

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[19:08:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
[19:08:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대로템
모델이름:  CatBoostClassifier
accuracy:0.1667, precision:0.9000, recall:0.1552, roc_auc:0.1293
./data/machine_model3_6개월_0.05/현대로템_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대리바트
회사이름:  현대리바트
모델이름:  RandomForestClassifier
accuracy:0.3167, precision:0.6000, recall:0.0714, roc_auc:0.4213
./data/machine_model3_6개월_0.05/현대리바트_RandomForestClassifier.pkl
--------------------------------------------------
[19:29:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대리바트
모델이름:  CatBoostClassifier
accuracy:0.3333, precision:0.7500, recall:0.0714, roc_auc:0.4649
./data/machine_model3_6개월_0.05/현대리바트_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대모비스
[19:30:01] WA

[19:30:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  혜인
모델이름:  XGBClassifier
accuracy:0.6667, precision:0.6471, recall:0.9429, roc_auc:0.6240
./data/machine_model3_6개월_0.05/혜인_XGBClassifier.pkl
--------------------------------------------------
회사이름:  혜인
모델이름:  CatBoostClassifier
accuracy:0.6667, precision:0.6471, recall:0.9429, roc_auc:0.5949
./data/machine_model3_6개월_0.05/혜인_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  호텔신라
회사이름:  호텔신라
모델이름:  RandomForestClassifier
accuracy:0.7167, precision:0.7907, recall:0.8095, roc_auc:0.5827
./data/machine_model3_6개월_0.05/호텔신라_RandomForestClassifier.pkl
--------------------------------------------------
[19:30:44] WARNING: C:/Users/Administrator/worksp

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화천기공
[19:30:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
회사이름:  황금에스티
모델이름:  RandomForestClassifier
accuracy:0.9000, precision:0.9818, recall:0.9153, roc_auc:0.9153
./data/machine_model3_6개월_0.05/황금에스티_RandomForestClassifier.pkl
--------------------------------------------------
[19:30:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
모델이름:  XGBClassifier
accuracy:0.9667, precision:0.9831, recall:0.9831, roc_auc:0.9576
./data/machine_model3_6개월_0.05/황금

In [12]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,BGF,RandomForestClassifier,0.583333,0.555556,0.535714,0.641741,./data/machine_model3_6개월_0.05/BGF_RandomForestClassifier.pkl
1,BGF,XGBClassifier,0.650000,0.666667,0.500000,0.583705,./data/machine_model3_6개월_0.05/BGF_XGBClassifier.pkl
2,BGF,CatBoostClassifier,0.616667,0.600000,0.535714,0.615513,./data/machine_model3_6개월_0.05/BGF_CatBoostClassifier.pkl
3,CJ CGV,RandomForestClassifier,0.483333,0.703704,0.452381,0.517857,./data/machine_model3_6개월_0.05/CJ CGV_RandomForestClassifier.pkl
4,CJ CGV,XGBClassifier,0.483333,0.739130,0.404762,0.650794,./data/machine_model3_6개월_0.05/CJ CGV_XGBClassifier.pkl
...,...,...,...,...,...,...,...
297,효성ITX,XGBClassifier,0.450000,0.666667,0.058824,0.484163,./data/machine_model3_6개월_0.05/효성ITX_XGBClassifier.pkl
298,효성ITX,CatBoostClassifier,0.466667,0.750000,0.088235,0.514706,./data/machine_model3_6개월_0.05/효성ITX_CatBoostClassifier.pkl
299,후성,RandomForestClassifier,0.716667,0.829268,0.772727,0.784801,./data/machine_model3_6개월_0.05/후성_RandomForestClassifier.pkl
300,후성,XGBClassifier,0.766667,0.916667,0.750000,0.803267,./data/machine_model3_6개월_0.05/후성_XGBClassifier.pkl


## 모든파일 만들기 6개월 10%

In [13]:
month = 6
window_size = 21*month
period_rate= 0.1

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-03-31',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-03-31','target']
        
        x_test = df_model.loc['2021-04-01':,:last_col]
        y_test = df_model.loc['2021-04-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[19:33:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[19:33:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[19:33:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
회사이름:  BYC
모델이름:  Ran

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[19:34:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[19:34:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[19:34:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[19:34:53] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  HJ중공업
[19:34:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
회사이름:  HMM
모델이름:  RandomForestClassifier
accuracy:0.7000, precision:0.9767, recall:0.7119, roc_auc:0.1356
./data/machine_model3_6개월_0.1/HMM_RandomForestClassifier.pkl
--------------------------------------------------
[19:34:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HMM
모델이름:  XGBClassifier
accuracy:0.5833, precision:0.9722, recall:0.5932, roc_auc:0.3220
./data/machine_model3_6개월_0.1/HMM_XGBClas

...

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대로템
모델이름:  CatBoostClassifier
accuracy:0.2500, precision:0.7500, recall:0.1224, roc_auc:0.4249
./data/machine_model3_6개월_0.1/현대로템_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대리바트
[19:56:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[19:56:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
회사이름:  현대미포조선
모델이름:  RandomForestClassifier
accuracy:0.4833, precision:0.9032, recall:0.5000, roc_auc:0.3661
./data/machine_mode

회사이름:  화승코퍼레이션
[19:57:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  화신
[19:57:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화천기공
[19:57:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
회사이름:  황금에스티
모델이름:  RandomForestClassifier
accuracy:0.0667, precision:0.6667, recall:0.0351, roc_auc:0.3275
./data/machine_model3_6개월_0.1/황금에스티_RandomForestClassifier.pkl
--------------------------------------------------
[19:57:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
모델이름:  CatBoostClassifier
accuracy:0.0667, precision:0.6667, recall:0.0351, roc_auc:0.3275
./data/machine_model3_6개월_0.1

In [14]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,BYC,RandomForestClassifier,0.133333,0.833333,0.089286,0.294643,./data/machine_model3_6개월_0.1/BYC_RandomForestClassifier.pkl
1,BYC,CatBoostClassifier,0.116667,0.714286,0.089286,0.294643,./data/machine_model3_6개월_0.1/BYC_CatBoostClassifier.pkl
2,DB,XGBClassifier,0.833333,0.980392,0.847458,0.830508,./data/machine_model3_6개월_0.1/DB_XGBClassifier.pkl
3,DI동일,RandomForestClassifier,0.416667,0.960000,0.413793,0.448276,./data/machine_model3_6개월_0.1/DI동일_RandomForestClassifier.pkl
4,DI동일,XGBClassifier,0.416667,0.960000,0.413793,0.387931,./data/machine_model3_6개월_0.1/DI동일_XGBClassifier.pkl
...,...,...,...,...,...,...,...
186,혜인,XGBClassifier,0.566667,0.714286,0.172414,0.633482,./data/machine_model3_6개월_0.1/혜인_XGBClassifier.pkl
187,혜인,CatBoostClassifier,0.533333,0.555556,0.172414,0.615128,./data/machine_model3_6개월_0.1/혜인_CatBoostClassifier.pkl
188,황금에스티,RandomForestClassifier,0.066667,0.666667,0.035088,0.327485,./data/machine_model3_6개월_0.1/황금에스티_RandomForestClassifier.pkl
189,황금에스티,CatBoostClassifier,0.066667,0.666667,0.035088,0.327485,./data/machine_model3_6개월_0.1/황금에스티_CatBoostClassifier.pkl


## 모든파일 만들기 6개월 15%

In [15]:
month = 6
window_size = 21*month
period_rate= 0.15

path = './data/데이터_뉴스키워드빈도/'
file_list = os.listdir(path)

corp_list =[]
model_list = []
accu_list = []
precision_list = []
recall_list = []
roc_list = []
model_path = []

for file in file_list[:]:
    corp_name = file[:-4]
    print('회사이름: ',corp_name)
    code = corp_code(corp_name)
    df_p = stock_price(code)
    
    file_path = os.path.join(path,file)
    df_count = pd.read_csv(file_path,index_col=0)
    df_count.index = pd.DatetimeIndex(df_count.index)
    try:
        last_col = df_count.columns[-1]
    except:
        continue
    
    # 5개 미만 단어 아예없는경우는 생략
    try:
        df_count = mscaler(df_count)

        df_merge = merge(df_count,df_p)

        df_model = add_target(df_merge,window_size=window_size, period_rate= period_rate)

        x_train = df_model.loc['2016-05-02':'2020-12-31',:last_col]
        y_train = df_model.loc['2016-05-02':'2020-12-31','target']
        
        x_val = df_model.loc['2021-01-04':'2021-03-31',:last_col]
        y_val = df_model.loc['2021-01-04':'2021-03-31','target']
        
        x_test = df_model.loc['2021-04-01':,:last_col]
        y_test = df_model.loc['2021-04-01':,'target']


        for model in models :
            time.sleep(0.3)
            model_name = model.__class__.__name__

            try:

                if model_name == 'CatBoostClassifier':
                    model.fit(x_train,y_train, silent=True)
                else:
                    model.fit(x_train,y_train)

            except:
                continue

            pred = model.predict(x_val)
            accuracy = accuracy_score(y_val, pred)
            proba = model.predict_proba(x_val)[:,1]
            precision = precision_score(y_val, pred)
            recall = recall_score(y_val, pred)

            try:
                roc_auc = roc_auc_score(y_val, proba)
            except:
                roc_auc = 0

            save_path = f'./data/machine_model3_{month}개월_{period_rate}/{corp_name}_{model_name}.pkl'


            if (precision > 0.5) & (precision!=1) &(recall!=1):

                corp_list.append(corp_name)
                model_list.append(model_name)
                accu_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                roc_list.append(roc_auc)
                model_path.append(save_path)
                joblib.dump(model, save_path)

                print('회사이름: ',corp_name)
                print('모델이름: ',model_name)
                print('accuracy:{0:.4f}, precision:{1:.4f}, recall:{2:.4f}, roc_auc:{3:.4f}'.format(accuracy,precision,recall,roc_auc))
                print(save_path)
                print('-'*50)

    except:
        print('출현 단어 5개 미만 종목')
    
    print('='* 50)
    
df_result = pd.DataFrame({'회사이름':corp_list, '모델이름':model_list,'accuracy':accu_list,
                          'precision':precision_list,'recall':recall_list,'roc_auc':roc_list,'모델주소':model_path})

df_result.to_csv(f'./data/model_result_val/machine_model3_{window_size}일_{period_rate}.csv')

회사이름:  AJ네트웍스
[19:58:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  AK홀딩스
[19:58:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BGF
[19:58:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  BYC
회사이름:  BYC
모델이름:  Ran

회사이름:  GS건설
[19:59:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  GS글로벌
[19:59:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  GS리테일
[19:59:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC
[19:59:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  HDC랩스
[19:59:46] WARNING: 

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

...

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  현대로템
모델이름:  CatBoostClassifier
accuracy:0.3000, precision:0.6000, recall:0.0698, roc_auc:0.4118
./data/machine_model3_6개월_0.15/현대로템_CatBoostClassifier.pkl
--------------------------------------------------
회사이름:  현대리바트
[20:21:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대모비스
[20:21:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  현대미포조선
회사이름:  현대미포조선
모델이름:  RandomForestClassifier
accuracy:0.4833, precision:0.9032, recall:0.5000, roc_auc:0.3549
./data/machine_mod

회사이름:  화신
[20:22:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Custom logger is already specified. Specify more than one logger at same time is not thread safe.

회사이름:  화천기공
[20:22:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
회사이름:  황금에스티
모델이름:  RandomForestClassifier
accuracy:0.1333, precision:0.6667, recall:0.0377, roc_auc:0.5660
./data/machine_model3_6개월_0.15/황금에스티_RandomForestClassifier.pkl
--------------------------------------------------
[20:22:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
회사이름:  황금에스티
모델이름:  CatBoostClassifier
accuracy:0.1333, precision:0.6667, recall:0.0377, roc_auc:0.5633
./data/machine_model3_6개월_0.

In [16]:
df_result

,회사이름,모델이름,accuracy,precision,recall,roc_auc,모델주소
0,BYC,RandomForestClassifier,0.150000,0.571429,0.076923,0.352163,./data/machine_model3_6개월_0.15/BYC_RandomForestClassifier.pkl
1,BYC,CatBoostClassifier,0.150000,0.571429,0.076923,0.347356,./data/machine_model3_6개월_0.15/BYC_CatBoostClassifier.pkl
2,DB,RandomForestClassifier,0.833333,0.923077,0.888889,0.500000,./data/machine_model3_6개월_0.15/DB_RandomForestClassifier.pkl
3,DB,XGBClassifier,0.800000,0.920000,0.851852,0.537037,./data/machine_model3_6개월_0.15/DB_XGBClassifier.pkl
4,DB,CatBoostClassifier,0.833333,0.923077,0.888889,0.478395,./data/machine_model3_6개월_0.15/DB_CatBoostClassifier.pkl
...,...,...,...,...,...,...,...
155,혜인,RandomForestClassifier,0.550000,0.538462,0.250000,0.559152,./data/machine_model3_6개월_0.15/혜인_RandomForestClassifier.pkl
156,혜인,XGBClassifier,0.583333,0.714286,0.178571,0.666295,./data/machine_model3_6개월_0.15/혜인_XGBClassifier.pkl
157,혜인,CatBoostClassifier,0.550000,0.555556,0.178571,0.608259,./data/machine_model3_6개월_0.15/혜인_CatBoostClassifier.pkl
158,황금에스티,RandomForestClassifier,0.133333,0.666667,0.037736,0.566038,./data/machine_model3_6개월_0.15/황금에스티_RandomForestClassifier.pkl
